In [1]:
import json
from dl import ComicTextDetector, MangaOCR, LamaInpainterMPE, SugoiTranslator
from utils.io_utils import imread, imwrite, NumpyEncoder
import os
from pathlib import Path

test_img = r'data/testpacks/custom/07.png'
image_path = Path(test_img).expanduser().absolute()
img = imread(image_path)

test_dir = image_path.parent

mask_dir = os.path.join(test_dir, 'mask')
os.makedirs(mask_dir, exist_ok=True)

ocr_dir = os.path.join(test_dir, 'ocr_test')
os.makedirs(ocr_dir, exist_ok=True)

inpaint_dir = os.path.join(test_dir, 'inpainted')
os.makedirs(inpaint_dir, exist_ok=True)

ctd = ComicTextDetector()


def mangaocr(img, blk_list):
    ocr = MangaOCR()
    blk_list = ocr.run_ocr(img, blk_list)
    return blk_list


def inpaint_lama_mpe(img, mask, blk_list=None, inpaint_by_block=True):
    inpainter = LamaInpainterMPE()
    inpainter.inpaint_by_block = inpaint_by_block
    inpainted = inpainter.inpaint(img, mask, blk_list)
    # imwrite(inpaint_dir + '/' + image_path.name, inpainted)
    return inpainted


def sugoi_translate(text):
    translator = SugoiTranslator('日本語', 'English')
    translation = translator.translate(text)
    print(f'src: {text}, translation: {translation}')


mask, blk_list = ctd.detect(img)
blk_list = mangaocr(img, blk_list)
# inpainted = inpaint_lama_mpe(img, mask, blk_list)
# imwrite(mask_dir + '/' + image_path.name, mask)
# imwrite(inpaint_dir + '/' + image_path.name, inpainted)

# not working yet
# with open(image_path.with_suffix('.json'), 'w', encoding='utf-8') as f:
# f.write(json.dumps(blk_list, ensure_ascii=False, cls=NumpyEncoder))


# json.dumps(blk_list[0].__dict__, ensure_ascii=False, cls=NumpyEncoder)


/home/kys/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-22 08:26:49.361310: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
translator = SugoiTranslator('日本語', 'English')


In [4]:
blk_list = translator.translate_textblk_lst(blk_list)

In [2]:
blk_list[3].xyxy

[57, 1015, 164, 1315]

In [5]:
H, W, *_ = img.shape
result = {'img_width': W,
          'img_height': H, 
          'blocks': []
          }

for blk in blk_list:
    result_blk = {
        'text_box': blk.xyxy,
        'text_ocr': blk.text,
        'text_tl': blk.translation,
        'font_size': blk.font_size,
    }
    result['blocks'].append(result_blk)



In [6]:
result

{'img_width': 960,
 'img_height': 1361,
 'blocks': [{'text_box': [731, 86, 805, 239],
   'text_ocr': 'おい！古屋晴久！',
   'text_tl': ' Hey! Haruhisa Furuya!',
   'font_size': 31},
  {'text_box': [417, 400, 519, 580],
   'text_ocr': '桜！',
   'text_tl': ' Sakura!',
   'font_size': 97},
  {'text_box': [740, 1035, 814, 1277],
   'text_ocr': '昨日連絡した通り来てやったわよ。',
   'text_tl': ' I contacted you yesterday. I came through.',
   'font_size': 30},
  {'text_box': [57, 1015, 164, 1315],
   'text_ocr': 'あんたの監視役としてね。',
   'text_tl': ' As your watcher.',
   'font_size': 43}]}

In [10]:
from utils.json_writer import dump_json
dump_json(result, image_path.with_suffix('.json'))
